## Imports

In [75]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout,Flatten,Dense
from keras.models import Sequential

In [76]:
#read natis photos
# try to predict on the
%matplotlib inline

#import cv2
import matplotlib.pyplot as plt
from PIL import Image

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
datagen = ImageDataGenerator()
#from vgg16bn import Vgg16BN
import utils
import PIL
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.cross_validation import train_test_split
import subprocess
import cv2
from utils import *

import glob
import numpy as np

In [77]:
def limit_mem():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    
limit_mem()

In [78]:
def get_im_cv2(path, img_rows, img_cols, color_type=3):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

def load_train_2(img_rows, img_cols, color_type=3): # - with no outer loop
    X_train = []
    y_train = []

    print('Read train images')
    #for j in unique_fish_cats:
    #print('Load folder {}'.format(j))
    #path = os.path.join('*/*.jpg')
    #files = glob.glob(path)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl, img_rows, img_cols, color_type)

        X_train.append(img)
        j=fl.split('/')[0]
        y_train.append(j)

    return X_train, y_train


## read data

In [79]:
%cd /mnt/kaggle_fisherman/preprocessed_train/

/mnt/kaggle_fisherman/preprocessed_train


In [80]:

files=glob.glob('*.jpg')

In [81]:
labels=[name.split('_')[3].split('.')[0] for name in files]
img_ids=[name.split('.')[0] for name in files]

In [82]:
from collections import Counter
Counter(labels)

Counter({'ALB': 1711,
         'BET': 199,
         'DOL': 117,
         'LAG': 67,
         'OTHER': 299,
         'SHARK': 175,
         'YFT': 731})

In [83]:
#read images
im_list=[]
for file in files:
    im_list.append(get_im_cv2(file,224,224,3))
im_array=np.array(im_list)
label_ids=[]
unique_labels=np.unique(labels)
for label in labels:
    label_ids.append(np.where(unique_labels==label)[0][0])

In [84]:
im_list[0].shape,len(im_list)

((224, 224, 3), 3299)

### standard train test split

In [85]:
X_trn,X_val,y_trn,y_val=train_test_split(im_array,np.array(label_ids),test_size=0.1)

### logo split

In [17]:
# this part is taken from anokas's boat clustering script

import pandas as pd
import numpy as np
import glob
from sklearn import cluster
from scipy.misc import imread
import cv2
import skimage.measure as sm
# import progressbar
import multiprocessing
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
new_style = {'grid': False}
plt.rc('axes', **new_style)

# Function to show 4 images
def show_four(imgs, title):
    #select_imgs = [np.random.choice(imgs) for _ in range(4)]
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(4)]
    _, ax = plt.subplots(1, 4, sharex='col', sharey='row', figsize=(20, 3))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i].imshow(img)

# Function to show 8 images
def show_eight(imgs, title):
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(8)]
    _, ax = plt.subplots(2, 4, sharex='col', sharey='row', figsize=(20, 6))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i // 4, i % 4].imshow(img)

#select = 500 # Only load 500 images for speed
# Data loading
train_files = files
temp_train = np.array([imread(img) for img in train_files])
print('Length of train {}'.format(len(temp_train)))

# Function for computing distance between images
def compare(args):
    img, img2 = args
    img = (img - img.mean()) / img.std()
    img2 = (img2 - img2.mean()) / img2.std()
    return np.mean(np.abs(img - img2))

# Resize the images to speed it up.
temp_train = [cv2.resize(img, (224, 224), cv2.INTER_LINEAR) for img in temp_train]

# Create the distance matrix in a multithreaded fashion
pool = multiprocessing.Pool(12)
#bar = progressbar.ProgressBar(max=len(train))
distances = np.zeros((len(temp_train), len(temp_train)))
for i, img in enumerate(temp_train): #enumerate(bar(train)):
    all_imgs = [(img, f) for f in temp_train]
    dists = pool.map(compare, all_imgs)
    distances[i, :] = dists

cls = cluster.DBSCAN(metric='precomputed', min_samples=5, eps=0.6)
boat_ids = cls.fit_predict(distances)



Length of train 3299


In [112]:
ls

ALB/                           img_03979.jpg_label_OTHER.jpg
BET/                           img_03983.jpg_label_SHARK.jpg
DOL/                           img_03984.jpg_label_ALB.jpg
LAG/                           img_03985.jpg_label_ALB.jpg
OTHER/                         img_03990.jpg_label_ALB.jpg
SHARK/                         img_03991.jpg_label_ALB.jpg
YFT/                           img_03997.jpg_label_DOL.jpg
boat_ids.csv                   img_03999.jpg_label_YFT.jpg
boat_ids_correct.csv           img_04005.jpg_label_ALB.jpg
img_00003.jpg_label_ALB.jpg    img_04006.jpg_label_ALB.jpg
img_00004.jpg_label_YFT.jpg    img_04008.jpg_label_ALB.jpg
img_00010.jpg_label_ALB.jpg    img_04011.jpg_label_ALB.jpg
img_00012.jpg_label_ALB.jpg    img_04012.jpg_label_OTHER.jpg
img_00014.jpg_label_YFT.jpg    img_04014.jpg_label_ALB.jpg
img_00015.jpg_label_ALB.jpg    img_04016.jpg_label_ALB.jpg
img_00019.jpg_label_ALB.jpg    img_04018.jpg_label_ALB.jpg
img_00020.jpg_label_ALB.jpg    img_04021.jpg_label

In [110]:
boat_df=pd.read_csv('boat_ids.csv')

In [13]:
boat_list=boat_df['0'].values

In [113]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()
logo.get_n_splits(im_array, label_ids, boat_list)

50

In [114]:
i=1
stop=1
for train_index, test_index in logo.split(im_array, label_ids, boat_list):
    X_trn, X_val, y_trn,y_val  = im_array[train_index], im_array[test_index], np.array(label_ids)[train_index], np.array(label_ids)[test_index]
    if i==stop:
        break
    i+=1

In [115]:
X_trn.shape

(2313, 224, 224, 3)

logo split ends here

## build models

### standard VGG

In [116]:
def split_at(model, layer_type):
    layers = model.layers
    layer_idx = [index for index,layer in enumerate(layers) if type(layer) is layer_type][-1]
    return layers[:layer_idx+1], layers[layer_idx+1:]

In [138]:
model=VGG16()

In [118]:
conv_layers,fc_layers = split_at(model, Convolution2D)

In [139]:
for i in  range(5):
    model.layers.pop() # i did it 5 time

In [140]:
# bn layers - all of this is to prevent dividing the model, because of the generator
p=0.5
#inp=Input ((224,224,3))
x = model.layers[-1].output
x = MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:])(x)
x=BatchNormalization(axis=1)(x)
x=Dropout(p/4)(x)
x=Flatten()(x)
x=Dense(512, activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(p)(x)
x=Dense(512, activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(p/2)(x)
x=Dense(7, activation='softmax')(x)


In [141]:
model1 = Model(input = model.input,output=[x])

In [142]:
for i,layer in enumerate(model1.layers):
    if i<18:
        model1.layers[i].trainable = False

### fcn

In [146]:
nf=128; p=0.

In [147]:
#fcn layers
x = model.layers[-1].output
x=BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:])(x)
x=Convolution2D(nf,3,3, activation='relu', border_mode='same')(x)
x=BatchNormalization(axis=1)(x)
x=MaxPooling2D()(x)
x=Convolution2D(nf,3,3, activation='relu', border_mode='same')(x)
x=BatchNormalization(axis=1)(x)
x=MaxPooling2D()(x)
x=Convolution2D(nf,3,3, activation='relu', border_mode='same')(x)
x=BatchNormalization(axis=1)(x)
x=MaxPooling2D((1,2))(x)
x=Convolution2D(7,3,3, border_mode='same')(x)
x=Dropout(p)(x)
x=GlobalAveragePooling2D()(x)
x=Activation('softmax')(x)

In [148]:
model2 = Model(input = model.input,output=[x])

In [149]:
for i,layer in enumerate(model2.layers):
    if i<18:
        model2.layers[i].trainable = False

In [152]:
len(model2.layers),len(model1.layers)

(32, 29)

### Inception

In [155]:
from keras.applications.resnet50 import ResNet50

In [179]:
resnet=ResNet50()

In [183]:
a=resnet.layers.pop()

In [188]:
len(resnet.layers)

177

In [187]:
x = resnet.layers[-1].output
x=Dense(7, activation='softmax')(x)
resnet = Model(input = resnet.input,output=[x])

In [181]:

for i,layer in enumerate(resnet.layers):
    if i<173:
        layer.trainable=False


In [173]:
for layer in resnet.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

### training

In [143]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=180,
    rescale=2,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True
)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
#datagen.fit(X_trn)


In [144]:
model1.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
#conv_val_feat = conv_model.predict_generator(X_val,batch_size=16)

In [136]:
model1.fit(X_trn, to_categorical(y_trn), batch_size=32 ,validation_data=(X_val,to_categorical(y_val)))

Train on 2313 samples, validate on 986 samples
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
model1.fit(X_trn, to_categorical(y_trn), batch_size=32 ,validation_data=(X_val,to_categorical(y_val)))

In [145]:
model1.fit_generator(datagen.flow(X_trn, to_categorical(ytrain), batch_size=32)\
                     ,validation_data=datagen.flow(X_val, to_categorical(y_val)) \
                     ,samples_per_epoch=3000,nb_val_samples=16,nb_epoch=10)

Epoch 1/10


/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:464: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:472: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2985/3000 [============================>.] - ETA: 0s - loss: 1.3963 - acc: 0.5816  

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3017/3000 [==============================] - 70s - loss: 1.3930 - acc: 0.5830 - val_loss: 6.7633 - val_acc: 0.3750
Epoch 2/10
3017/3000 [==============================] - 68s - loss: 0.7749 - acc: 0.7673 - val_loss: 1.3503 - val_acc: 0.6562

KeyboardInterrupt: 

In [153]:
model2.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [129]:
model2.fit(X_trn, to_categorical(y_trn), batch_size=32 ,validation_data=(X_val,to_categorical(y_val)))

Train on 2313 samples, validate on 986 samples
Epoch 1/10
2313/2313 [==============================] - 96s - loss: 1.0485 - acc: 0.6684 - val_loss: 1.5661 - val_acc: 0.5710

KeyboardInterrupt: 

In [154]:
model2.fit_generator(datagen.flow(X_trn, to_categorical(ytrain), batch_size=32)\
                     ,validation_data=datagen.flow(X_val, to_categorical(y_val)) \
                     ,samples_per_epoch=3000,nb_val_samples=16,nb_epoch=10)

Epoch 1/10


/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:464: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:472: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2985/3000 [============================>.] - ETA: 0s - loss: 0.9649 - acc: 0.6683  

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3017/3000 [==============================] - 94s - loss: 0.9640 - acc: 0.6685 - val_loss: 2.2618 - val_acc: 0.6250
Epoch 2/10
3017/3000 [==============================] - 92s - loss: 0.5789 - acc: 0.7952 - val_loss: 1.5382 - val_acc: 0.6875

KeyboardInterrupt: 

In [189]:
resnet.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [193]:
resnet.fit_generator(datagen.flow(X_trn, to_categorical(y_trn), batch_size=32)\
                     ,validation_data=datagen.flow(X_val, to_categorical(y_val)) \
                     ,samples_per_epoch=3000,nb_val_samples=len(y_val),nb_epoch=10)

Epoch 1/10


/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:464: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:472: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2985/3000 [============================>.] - ETA: 0s - loss: 0.3170 - acc: 0.9002 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3017/3000 [==============================] - 51s - loss: 0.3172 - acc: 0.8999 - val_loss: 1.0214 - val_acc: 0.6927
Epoch 2/10
3017/3000 [==============================] - 50s - loss: 0.3171 - acc: 0.8896 - val_loss: 0.9987 - val_acc: 0.6907

KeyboardInterrupt: 

In [1]:
#write models
model_json = model1.to_json()
model_path='/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_united_test.json'
with open(model_path, "w") as json_file:
    json_file.write(model_json)
    
#model1.save_weights('/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_united.hdf5')

NameError: name 'model1' is not defined

## stuff ends here

In [68]:
#write models
# model_json = bn_model.to_json()
# model_path='/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_0.json'
# with open(model_path, "w") as json_file:
#     json_file.write(model_json)
    
# model_json = conv_model.to_json()
# model_path='/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_pre.json'
# with open(model_path, "w") as json_file:
#     json_file.write(model_json)

# bn_model.save_weights('/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_0.hdf5')
# conv_model.save_weights('/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_pre.hdf5')

In [69]:
# #load models

# from keras.models import model_from_json

# model_path='/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_pre.json'
# json_file = open(model_path, 'r')
# model_classifier_0 = model_from_json(json_file.read())
# json_file.close()
# model_classifier_0.load_weights('/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_pre.hdf5', by_name=True)


# model_path='/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_0.json'
# json_file = open(model_path, 'r')
# model_classifier_1 = model_from_json(json_file.read())
# json_file.close()
# model_classifier_1.load_weights('/mnt/kaggle_fisherman/cropped_fish_models/cropped_fish_model_0.hdf5', by_name=True)

In [70]:
pred=model_classifier_0.predict(X_val)
pred=model_classifier_1.predict(pred)

In [63]:
pred=bn_model.predict(conv_val_feat)

In [71]:
from sklearn.metrics import log_loss
from keras.utils.np_utils import to_categorical

log_loss(y_val,pred)


0.121530299022338

In [42]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx) 
pred = do_clip(pred,0.82)

In [43]:
%cd /datadrive/kaggle_fisherman/train

/datadrive/kaggle_fisherman/train


In [51]:
#get list of image sizes
import PIL
path = "/datadrive/kaggle_fisherman/"
sizes = [PIL.Image.open(path+'train/'+f).size for f in train_file_names]
id2size = list(set(sizes))
size2id = {o:i for i,o in enumerate(id2size)}

In [47]:
train_file_names = glob.glob('*/*.jpg')
fish_cats=[i.split('/')[0] for i in train_file_names]
unique_fish_cats=np.unique(fish_cats)

In [59]:
id_to_size_dict={str(int(file.split('_')[1].split('.')[0])):size for file,size in zip(train_file_names,sizes)}

## submit

In [293]:
#load test files
%cd ../cropped_test_preds/

/mnt/kaggle_fisherman/cropped_test_preds


In [197]:
t=glob.glob('*.*')

In [296]:
len(t)

60765

In [202]:
t[:10][0].split('.')

['image_10403', 'jpg_3_0', '465505808592', 'npy']

In [302]:
test=[]
for file in t:
    img = np.load(file)
    test.append(img)

In [303]:
test_array=np.array(test)

In [304]:
test_names=[file.split('.')[0]+'.jpg' for file in t]

In [305]:
probs=[file.split('_')[-1][:-4] for file in t]

In [306]:
preds=resnet.predict(test_array[:,0,:,:,:]) #add tqdm here...

In [307]:
prob_dic={}
pred_dic={}
for name in test_names:
    prob_dic[name]=[]
    pred_dic[name]=[]
    
for i,name in enumerate(test_names):
    prob_dic[name].append(float(probs[i]))
    pred_dic[name].append(preds[i,:])

pred_list=[]
prob_list=[]

unique_names=np.unique(test_names)
for i,name in enumerate(unique_names):
    pred_list.append(pred_dic[name])
    prob_list.append(prob_dic[name])

In [308]:
#temporary addition for stage1
new_preds_1=[]
for p,prob in zip(pred_list,prob_list):
    #print 
    new_preds_1.append(np.average(p,weights=prob,axis=0))
new_preds_1=np.array(new_preds_1)
new_preds_1=np.insert(new_preds_1, [4], np.array([0.85-np.max(prob_list,axis=1)]).T, axis=1) #NOF!
new_preds_1 = do_clip(new_preds_1,0.82)

In [284]:
new_preds=[]
for p,prob in zip(pred_list,prob_list):
    #print 
    new_preds.append(np.average(p,weights=prob,axis=0))
new_preds=np.array(new_preds)
new_preds=np.insert(new_preds, [4], np.array([0.85-np.max(prob_list,axis=1)]).T, axis=1) #NOF!
new_preds = do_clip(new_preds,0.82)

In [311]:
new_preds.shape,new_preds_1.shape,len(unique_names)

((12153, 8), (1000, 8), 1000)

In [288]:
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(new_preds, columns=classes)
submission.insert(0, 'image', unique_names)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,image_00001.jpg,0.215343,0.085732,0.085732,0.085732,0.201707,0.154290,0.085732,0.085732
1,image_00002.jpg,0.343436,0.088254,0.088254,0.088254,0.127042,0.088254,0.088254,0.088254
2,image_00003.jpg,0.302254,0.088034,0.147252,0.088034,0.110325,0.088034,0.088034,0.088034
3,image_00004.jpg,0.188569,0.112121,0.112121,0.112121,0.113303,0.112121,0.126198,0.123444
4,image_00005.jpg,0.336591,0.091774,0.091774,0.091774,0.091774,0.112766,0.091774,0.091774


In [338]:
new_names=[]
for name in list(full_sub['image']):
    if name.split('_')[0]=='image':
         new_names.append('test_stg2/'+name)
    else:
        new_names.append(name)
        

In [339]:
full_sub['image']=new_names

In [340]:
full_sub.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.215343,0.085732,0.085732,0.085732,0.201707,0.154290,0.085732,0.085732
1,test_stg2/image_00002.jpg,0.343436,0.088254,0.088254,0.088254,0.127042,0.088254,0.088254,0.088254
2,test_stg2/image_00003.jpg,0.302254,0.088034,0.147252,0.088034,0.110325,0.088034,0.088034,0.088034
3,test_stg2/image_00004.jpg,0.188569,0.112121,0.112121,0.112121,0.113303,0.112121,0.126198,0.123444
4,test_stg2/image_00005.jpg,0.336591,0.091774,0.091774,0.091774,0.091774,0.112766,0.091774,0.091774


In [290]:
len(submission)

12153

In [341]:
full_sub.to_csv('../stage2_resnet', index=False)